<a href="https://colab.research.google.com/github/belvdere/NeuralNetworkAssignment/blob/main/Belvedere_Song_Zheng_Yi_Part_B_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question B4 (10 marks)

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
!pip install alibi-detect
!pip install pytorch_tabular[extra]

In [17]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig
)

1.Evaluate your model from B1 on data from year 2022 and report the test R2.

In [4]:
df = pd.read_csv('hdb_price_prediction.csv')
df = df.drop(['full_address', 'nearest_stn'], axis=1)

train_df = df[df.year < 2020]
val_df = df[df.year == 2020]
test_df_2022 = df[df.year == 2022]
test_df_2023 = df[df.year == 2023]


continuous_vars = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
categorical_vars = [ "month", "town", "flat_model_type", "storey_range"]

test_df_2022

,month,year,town,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
116427,1,2022,ANG MO KIO,0.313515,7.603794,0.016807,0.006243,"2 ROOM, Improved",54.416667,44.0,07 TO 09,245000.0
116428,1,2022,ANG MO KIO,0.508074,7.350360,0.016807,0.006243,"3 ROOM, New Generation",54.416667,73.0,07 TO 09,358000.0
116429,1,2022,ANG MO KIO,1.305684,7.592066,0.016807,0.006243,"3 ROOM, New Generation",55.083333,67.0,07 TO 09,355000.0
116430,1,2022,ANG MO KIO,0.884872,6.981730,0.016807,0.006243,"3 ROOM, New Generation",58.000000,68.0,07 TO 09,338000.0
116431,1,2022,ANG MO KIO,0.677246,8.333056,0.016807,0.006243,"3 ROOM, New Generation",57.166667,82.0,07 TO 09,420000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
143124,12,2022,YISHUN,1.187960,13.958828,0.016807,0.000382,"EXECUTIVE, Maisonette",65.000000,146.0,07 TO 09,845000.0
143125,12,2022,YISHUN,0.656870,13.540510,0.016807,0.000968,"EXECUTIVE, Apartment",63.833333,142.0,07 TO 09,782000.0
143126,12,2022,YISHUN,0.656870,13.540510,0.016807,0.000968,"EXECUTIVE, Apartment",63.833333,142.0,07 TO 09,770888.0
143127,12,2022,YISHUN,0.569011,13.549528,0.016807,0.000968,"EXECUTIVE, Maisonette",64.000000,146.0,10 TO 12,788000.0


In [6]:
data_config_2022 = DataConfig(
    target=['resale_price'],
    continuous_cols=continuous_vars,
    categorical_cols=categorical_vars,
)

trainer_config_2022 = TrainerConfig(
    auto_lr_find=True,
    batch_size=1024,
    max_epochs=50,
)

model_config_2022 = CategoryEmbeddingModelConfig(
    task="regression",
    layers=50,
)

optimizer_2022 = OptimizerConfig()

tab_model_2022 = TabularModel(
    data_config=data_config_2022,
    model_config=model_config_2022,
    optimizer_config=optimizer_2022,
    trainer_config=trainer_config_2022,
)

tab_model_2022.fit(train=train_df, validation=val_df)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67:

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_0b538889-4a81-4291-a9ca-e59bb612c2f0.ckpt
/usr/local/lib/python3.10/dist-packages/lightning_fabric/utilities/cloud_io.py:56: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by th

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 K                                                                                                
Total estimated model params size (MB): 0

Output()

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [7]:
result_2022 = tab_model_2022.evaluate(test_df_2022)
pred_df_2022 = tab_model_2022.predict(test_df_2022)

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       16876772352.0       │
│  test_mean_squared_error  │       16876772352.0       │
└───────────────────────────┴───────────────────────────┘

/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

2.Evaluate your model from B1 on data from year 2023 and report the test R2.

In [8]:
pred_df_2022

,resale_price_prediction
116427,194151.00000
116428,323368.15625
116429,262471.50000
116430,308929.31250
116431,330179.56250
...,...
143124,588981.31250
143125,579871.68750
143126,579871.68750
143127,635772.06250


In [9]:
merged_df_2022 = pred_df_2022.merge(test_df_2022[['resale_price']], left_index=True, right_index=True, how='left')
merged_df_2022


,resale_price_prediction,resale_price
116427,194151.00000,245000.0
116428,323368.15625,358000.0
116429,262471.50000,355000.0
116430,308929.31250,338000.0
116431,330179.56250,420000.0
...,...,...
143124,588981.31250,845000.0
143125,579871.68750,782000.0
143126,579871.68750,770888.0
143127,635772.06250,788000.0


In [10]:
from sklearn.metrics import r2_score

print(result_2022[0]['test_mean_squared_error'])
mse = np.sqrt(float(result_2022[0]['test_mean_squared_error']))

prediction_2022 = pred_df_2022["resale_price_prediction"].values.tolist()
y_2022 = merged_df_2022['resale_price'].values.tolist()

r_squared = r2_score(y_2022, prediction_2022)
print("Test loss:", result_2022[0]['test_loss'])
print("RMSE", mse)
print("R Squared:", r_squared)

16876772352.0
Test loss: 16876772352.0
RMSE 129910.63217458378
R Squared: 0.41781369482948205


In [11]:
data_config_2023 = DataConfig(
    target=['resale_price'],
    continuous_cols=continuous_vars,
    categorical_cols=categorical_vars,
)

trainer_config_2023 = TrainerConfig(
    auto_lr_find=True,
    batch_size=1024,
    max_epochs=50,
)

model_config_2023 = CategoryEmbeddingModelConfig(
    task="regression",
    layers=50,
)

optimizer_2023 = OptimizerConfig()

tab_model_2023 = TabularModel(
    data_config=data_config_2023,
    model_config=model_config_2023,
    optimizer_config=optimizer_2023,
    trainer_config=trainer_config_2023,
)

tab_model_2023.fit(train=train_df, validation=val_df)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67:

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_a08c2d31-a6ee-461b-b4a7-f2826bba8777.ckpt
/usr/local/lib/python3.10/dist-packages/lightning_fabric/utilities/cloud_io.py:56: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by th

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 K                                                                                                
Total estimated model params size (MB): 0

Output()

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [12]:
result_2023 = tab_model_2023.evaluate(test_df_2023)
pred_df_2023 = tab_model_2023.predict(test_df_2023)

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       25508872192.0       │
│  test_mean_squared_error  │       25508872192.0       │
└───────────────────────────┴───────────────────────────┘

/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value



---



---



In [13]:
merged_df_2023 = pred_df_2023.merge(test_df_2023[['resale_price']], left_index=True, right_index=True, how='left')
merged_df_2023

,resale_price_prediction,resale_price
143129,130609.890625,267000.0
143130,185590.421875,300000.0
143131,173730.828125,280000.0
143132,187639.843750,282000.0
143133,128456.687500,289800.0
...,...,...
159548,558200.562500,780000.0
159549,613599.187500,808000.0
159550,605988.312500,788888.0
159551,533540.625000,822800.0


In [14]:
print(result_2023[0]['test_mean_squared_error'])
mse = np.sqrt(float(result_2023[0]['test_mean_squared_error']))

prediction_2023 = pred_df_2023["resale_price_prediction"].values.tolist()
y_2023 = merged_df_2023['resale_price'].values.tolist()

r_squared = r2_score(y_2023, prediction_2023)
print("Test loss:", result_2023[0]['test_loss'])
print("RMSE", mse)
print("R Squared:", r_squared)

25508872192.0
Test loss: 25508872192.0
RMSE 159714.97172150144
R Squared: 0.13473513715051932


3.Did model degradation occur for the deep learning model?

In [ ]:
# YOUR CODE HERE

Model degradation did occurred since R Sqaured dropped.

R Sqaured shows how well the independent variables explain the variability of the dependent variable. It is a statistical measure that indicates the proportion of the variance in the dependent variable that is predictable from the independent variables(s) in a regression model. A higher R Sqaured value suggests a better fit of the model to the data.

On test data 2023. R Squared = 0.135
On test data 2022, R Sqaured = 0.419

The dropped in R Sqaured shows that Model degradation did occured.

4.Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [21]:
# Get 1000 sample points
sample_train_df = train_df.sample(n=1000, random_state=1)
sample_test_df_2023 = test_df_2023.sample(n=1000, random_state=1)

categories_per_feature = {}

for cat in categorical_vars:
    categories_per_feature[sample_train_df.columns.get_loc(cat)] = None

In [22]:
cd = TabularDrift(sample_train_df.to_numpy(), p_val=.05, categories_per_feature=categories_per_feature)
pred = cd.predict(sample_test_df_2023.to_numpy(), drift_type='feature')

labels = ['No!', 'Yes!']

print(f"data:")
for key, val in pred['data'].items():
    print(f"{key} :{val}")

for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = sample_train_df.columns[f]
    is_drift = pred['data']['is_drift'][f]
    stat_val, p_val = pred['data']['distance'][f], pred['data']['p_val'][f]
    print(f'{fname:25s} -- Drift? {labels[is_drift]:5s} -- {stat:5s}: {stat_val:8.3f} -- p-value: {p_val:6.3f}')

data:
is_drift :[1 1 1 1 1 0 1 1 1 0 0 1]
distance :[4.6818979e+02 1.0000000e+00 4.3907768e+01 9.8999999e-02 8.9000002e-02
 3.0999999e-02 7.0000000e-02 6.6859573e+01 1.4399999e-01 4.6999998e-02
 1.6581758e+01 4.0300000e-01]
p_val :[0.0000000e+00 0.0000000e+00 1.1105743e-02 1.0194049e-04 6.7703531e-04
 7.1045446e-01 1.4168320e-02 9.3819859e-04 1.6429738e-09 2.1256767e-01
 2.7915153e-01 0.0000000e+00]
threshold :0.05
month                     -- Drift? Yes!  -- Chi2 :  468.190 -- p-value:  0.000
year                      -- Drift? Yes!  -- K-S  :    1.000 -- p-value:  0.000
town                      -- Drift? Yes!  -- Chi2 :   43.908 -- p-value:  0.011
dist_to_nearest_stn       -- Drift? Yes!  -- K-S  :    0.099 -- p-value:  0.000
dist_to_dhoby             -- Drift? Yes!  -- K-S  :    0.089 -- p-value:  0.001
degree_centrality         -- Drift? No!   -- K-S  :    0.031 -- p-value:  0.710
eigenvector_centrality    -- Drift? Yes!  -- K-S  :    0.070 -- p-value:  0.014
flat_model_type      

5.Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


In [ ]:
# YOUR CODE HERE

Type of data distribution shift possibly led to model degradation is Concept Drift.

Concept Drift occurs when the relationship between the input features (X) and the taret variable (Y) changes over time. This results P(X|Y) to change even when the features themselves remain the same.

This shift can degrade the model's performance because the model was trained based on an old relationship that is no longer valid.

This is based on the assumption that flurry of housing measures have made an impact on the relationship between all the features and resale_prices.

6.From your analysis via TabularDrift, which features contribute to this shift?


In [ ]:
# YOUR CODE HERE

The feature contribute to this shift is:
- month
- town
- dist_to_nearest_stn
- dist_to_dhoby
- eigenvector_centrality    
- flat_model_type
- remaining_lease_years

7.Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


In [ ]:
# YOUR CODE HERE

To address model degradation, since we identify that model degradation due to Concept Drift. We can improve the learning of the model to be able to predict future housing prices by feeding more data on the recent years into the model. This allow the model to learn the change in pattern over the years.

In [24]:
# include 2021 the new training samples
train_new_df = df[df.year < 2022]
val_new_df = df[df.year == 2022]
test_2023_new_df = df[df.year == 2023]

In [26]:
data_config_2023_new = DataConfig(
    target=['resale_price'],
    continuous_cols=continuous_vars,
    categorical_cols=categorical_vars,
)

trainer_config_2023_new = TrainerConfig(
    auto_lr_find=True,
    batch_size=1024,
    max_epochs=50,
)

model_config_2023_new = CategoryEmbeddingModelConfig(
    task="regression",
    layers=50,
)

optimizer_2023_new = OptimizerConfig()

tab_model_2023_new = TabularModel(
    data_config=data_config_2023_new,
    model_config=model_config_2023_new,
    optimizer_config=optimizer_2023_new,
    trainer_config=trainer_config_2023_new,
)

tab_model_2023_new.fit(train=train_new_df, validation=val_new_df)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67:

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_fcf60b34-4580-4719-85ab-75aa7026ff5f.ckpt
/usr/local/lib/python3.10/dist-packages/lightning_fabric/utilities/cloud_io.py:56: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by th

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 K                                                                                                
Total estimated model params size (MB): 0

Output()

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [27]:
result_2023_new = tab_model_2023_new.evaluate(test_2023_new_df)
pred_df_2023_new = tab_model_2023_new.predict(test_2023_new_df)

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       18126635008.0       │
│  test_mean_squared_error  │       18126635008.0       │
└───────────────────────────┴───────────────────────────┘

/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

In [28]:
merged_df_2023_new = pred_df_2023_new.merge(test_2023_new_df[['resale_price']], left_index=True, right_index=True, how='left')
merged_df_2023_new

,resale_price_prediction,resale_price
143129,165249.625000,267000.0
143130,200570.703125,300000.0
143131,192573.531250,280000.0
143132,199816.937500,282000.0
143133,175512.265625,289800.0
...,...,...
159548,558247.375000,780000.0
159549,611855.062500,808000.0
159550,604530.312500,788888.0
159551,549119.687500,822800.0


In [29]:
pred_2023_new = pred_df_2023_new["resale_price_prediction"].values.tolist()
y_2023_new = merged_df_2023_new['resale_price'].values.tolist()
r_squared_new = r2_score(y_2023_new, pred_2023_new)


print("MSE:", np.sqrt(float(result_2023_new[0]['test_mean_squared_error'])))
print("New R^2 Squared", r_squared_new)

MSE: 134635.19230869765
New R^2 Squared 0.38514169803960074


From the results,

Initial R Sqaured = 0.135 have increased to R Sqaured = 0.385.

This shows that the model now is better able to capture the pattern in the model and the change over time.

### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |